In [1]:
!pip install Trainer
!pip install accelerate -U
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.2/51.2 kB 2.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manyli

In [2]:
from transformers import CamembertTokenizer, CamembertForSequenceClassification
from transformers import Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
import pandas as pd

from sklearn.model_selection import train_test_split
from transformers import CamembertTokenizer, CamembertForSequenceClassification
from transformers import Trainer, TrainingArguments
from torch.utils.data import Dataset
import pandas as pd
import torch
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support
import numpy as np
import re

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
# Path to the CSV file
csv_path = '/content/drive/MyDrive/Colab Notebooks/df.csv'

df = pd.read_csv(csv_path)
df.head()

,id,title,text,url,lang,source_lang,created_at,published_at,rssfeed_id,source,...,reference_id,reference_key,keyword_list,location_list,publication_date,CLS_Lexique Sécurité Alimentaire,CLS_Relevance,CLS_Usefulness,CLS_Relevance Yes/No,CLS_Topic
0,HO8KNVZ6QF,Le plan d’autonomie est une “solution de compr...,Le plan d’autonomie est une “solution de compr...,https://www.24haubenin.info/?Le-plan-d-autonom...,FR,NaN,2022-10-26 21:25:51,2022-10-20 22:00:00+00:00,b5d565c4b5c4,www.24haubenin.info,...,NaN,NaN,NaN,NaN,1.666303e+09,no (m),No (m),No (m),No (m),Environement (m)
1,N7HP3S8B9V,Un quatuor béninois pour arbitrer Mali U23 vs ...,Un quatuor béninois pour arbitrer Mali U23 vs ...,https://www.24haubenin.info/?Un-quatuor-benino...,FR,NaN,2022-10-26 20:22:48,2022-10-20 22:00:00+00:00,b5d565c4b5c4,www.24haubenin.info,...,NaN,NaN,NaN,NaN,1.666303e+09,no (m),No (m),No (m),No (m),Sport (m)
2,ORG3BSXN7V,------------------- distinguée femme leader de...,------------------- distinguée femme leader de...,https://www.24haubenin.info/?Mariam-Chabi-Tala...,FR,NaN,2022-10-26 20:40:17,2022-10-20 22:00:00+00:00,b5d565c4b5c4,www.24haubenin.info,...,NaN,NaN,NaN,NaN,1.666303e+09,no (m),No (m),No (m),No (m),Sport (m)
3,ULGT4CHQHH,La CCI Bénin signe avec les CCI de Bahreïn et ...,La CCI Bénin signe avec les CCI de Bahreïn et ...,https://www.24haubenin.info/?La-CCI-Benin-sign...,FR,NaN,2022-10-26 20:48:53,2022-10-20 22:00:00+00:00,b5d565c4b5c4,www.24haubenin.info,...,NaN,NaN,NaN,NaN,1.666303e+09,no (m),No (m),No (m),No (m),Economie (m)
4,2HTTXAR4Q8,RADARISTES EST TROP !,RADARISTES EST TROP !\n\nOncle AGBAYA\nOn vous...,https://www.24haubenin.info/?RADARISTES-EST-TROP,FR,NaN,2022-10-26 20:31:19,2022-10-19 22:00:00+00:00,b5d565c4b5c4,www.24haubenin.info,...,NaN,NaN,NaN,NaN,1.666217e+09,no (m),No (m),No (m),No (m),Societe (m)


- ### 1
9IF61ML8VX
HX0YDP6SKD
INZW195KK6
MDQT5JR50I
VJ6T3D9O1F
D9E33KADWA
FPIOQTUII0
B2D7UMGZQB
IMYQMDG0NR
GBCYQXLO4R
OQBAG9MM3S
CO7C4BBM64
3U7IG5S8PW
NLAC4VLLMO
IRMEXP3AH4
2CWN59AFIE
JDIYQRPKVZ
5DUIUJFO6V
I9TZ78UR4X
W9KRXGMF1O
N0B2RZPZ24
VWWU1Y7XSZ
YOTY5INDM8
SQG850UO58
A9WMH4DX4R
3WQXVVV9B0
CREE6DQ7SR
JGVIRQHUAJ
J8QJLGZSSL
LMBTR5FE4U
903TVO65A0
ZF8RUVY4BJ
SY8NGUHVF7
SJ6IM28JF4
86FBX8GYLZ


- ### 0
Q5V3025UVM
NKOHJNYE2Q
DRQA2M008S
9U6JC1BFHF
OM2ANXEO3L
ZYNBMIIXV4
L9OWQXZ86J
Q78RDVRXC2
SY8NGUHVF7
C58CKT3Q58
AUGG57BNHJ
GAA0JNZRYN
C5ZIUGJLDF
Z3L9NKGUJF
4HKPS2AI7W
1K1557OT9K
B6WQNRRCR4
ZIN3LKR6YF
PB4JCP3MNL
13K7BZPILB
ZNF1R1LCQI
3DBXFCWK5C


In [47]:
ids_and_labels = {
    '1': [
        '9IF61ML8VX', 'HX0YDP6SKD', 'INZW195KK6', 'MDQT5JR50I', 'VJ6T3D9O1F', 'D9E33KADWA',
        'FPIOQTUII0', 'B2D7UMGZQB', 'C58CKT3Q58', 'GBCYQXLO4R', 'OQBAG9MM3S', 'CO7C4BBM64',
        '3U7IG5S8PW', 'NLAC4VLLMO', 'IRMEXP3AH4', '2CWN59AFIE', 'JDIYQRPKVZ', '5DUIUJFO6V',
        'I9TZ78UR4X', 'W9KRXGMF1O', 'N0B2RZPZ24', 'VWWU1Y7XSZ', 'YOTY5INDM8', 'SQG850UO58',
        'A9WMH4DX4R', '3WQXVVV9B0', 'CREE6DQ7SR', 'JGVIRQHUAJ', 'J8QJLGZSSL', 'LMBTR5FE4U',
        '903TVO65A0', 'Z3L9NKGUJF', 'SY8NGUHVF7', 'ZIN3LKR6YF', '86FBX8GYLZ', '09BSJ8CKZF',
        '1LG5V6E7SR', 'DCH6IOFHOR', 'DLG1X7JUQQ', 'D8A65D1BTK', 'IQVL0A9ZS3'
    ],
    '0': [
        'Q5V3025UVM', 'NKOHJNYE2Q', 'DRQA2M008S', '9U6JC1BFHF', 'OM2ANXEO3L', 'ZYNBMIIXV4',
        'L9OWQXZ86J', 'Q78RDVRXC2', 'FHQY1L4FZO', 'AUGG57BNHJ', 'GAA0JNZRYN',
        'C5ZIUGJLDF', 'N1FYCCQN8O', '4HKPS2AI7W', '1K1557OT9K', 'B6WQNRRCR4', 'RMM7WO52IO',
        'PB4JCP3MNL', '13K7BZPILB', 'ZNF1R1LCQI', '3DBXFCWK5C', 'IMYQMDG0NR', 'ZF8RUVY4BJ',
        'SJ6IM28JF4', 'DWZ3PIMFOX', 'P0G7VE8IX3', 'DW46I896M3', 'JUS9C00DF0', 'MZY4LO2YGH',
        '2ACMVMJ4R2', 'HVMMXFPMZR', 'SXL8FORGLY', 'HTCVEYFWZK', 'BR3VI3PQFJ', 'CHZLH5PUSO',
        'SVECPOCHY5', 'EP3A8RWV4Q', 'R785JXUBBD', 'PCSAMQ4IFY', 'KM8Q2VD9IC', 'IP9T1HMMK6']
}


In [56]:

extracted_data = []


for label, ids in ids_and_labels.items():
    for id in ids:

        row = df[df['id'] == id]
        if not row.empty:

            extracted_data.append({'id': id, 'label': label, 'text': row['text'].values[0]})


new_df = pd.DataFrame(extracted_data)


new_df.head(82)

,id,label,text
0,9IF61ML8VX,1,Avis de recrutement : Hommes et Terre recrute ...
1,HX0YDP6SKD,1,"Lutte contre le changement climatique : "" Nous..."
2,INZW195KK6,1,Le Sommet des Jeunes Futurs Leaders Africains ...
3,MDQT5JR50I,1,"Tribune | "" Vie chère : La menace qui vient du..."
4,VJ6T3D9O1F,1,Burkina Faso : Un atelier pour une transformat...
...,...,...,...
77,EP3A8RWV4Q,0,Homosexualité : la chasse aux sorcières électo...
78,R785JXUBBD,0,Que sommes nous devenus ?\n\nLettre ouverte d’...
79,PCSAMQ4IFY,0,A t-on une véritable politique touristique au ...
80,KM8Q2VD9IC,0,Non monsieur ------- vous ne devez nullement ê...


In [59]:
from transformers import CamembertForSequenceClassification, CamembertTokenizer

save_directory = "/content/drive/My Drive/MyModelMLM_2804"


model = CamembertForSequenceClassification.from_pretrained(save_directory)


In [60]:
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm

tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

class InferenceDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

texts = new_df['text'].tolist()

encodings = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors='pt')

inference_dataset = InferenceDataset(encodings)

dataloader = DataLoader(inference_dataset, batch_size=82)

model.eval()

model.to(model.device)


predicted_labels = []
confidence_scores = []

progress_bar = tqdm(dataloader, desc="Predicting")


with torch.no_grad():
    for batch in progress_bar:
        outputs = model(**{k: v.to(model.device) for k, v in batch.items()})
        logits = outputs.logits
        probs = torch.nn.functional.softmax(logits, dim=-1)
        top_probs, top_lbls = torch.max(probs, dim=1)
        predicted_labels.extend(top_lbls.cpu().numpy())
        confidence_scores.extend(top_probs.cpu().numpy())


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Predicting:   0%|          | 0/1 [00:00<?, ?it/s]

In [61]:
from sklearn.metrics import accuracy_score, precision_score, recall_score


label_mapping = {'0': 0, '1': 1}
new_df['label'] = new_df['label'].map(label_mapping)


accuracy = accuracy_score(new_df['label'], predicted_labels)

precision = precision_score(new_df['label'], predicted_labels)


recall = recall_score(new_df['label'], predicted_labels)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")

Accuracy: 0.8658536585365854
Precision: 0.8
Recall: 0.975609756097561


In [62]:
new_df['predicted label'] = predicted_labels

for idx, row in new_df.iterrows():
    text = row['text']
    id = row['id']
    predicted_label = row['predicted label']
    actual_label = row['label']

    if predicted_label != actual_label:
        print(f"Text: {text}")
        print(f"ID: {id}")
        print(f"Predicted Label: {predicted_label}, Actual Label: {actual_label}\n###############")

Text: Communique du conseil des ministres du mercredi 04 Mai 2022

Le --------------------------, ------------------------- a présidé le Conseil des Ministres, ce mercredi 04 mai 2022, au Palais de la République. A l’entame de sa communication, le Chef de l’Etat a présenté, à nouveau, ses meilleurs vœux à la communauté musulmane, à l’occasion de la célébration de l’Aïd al Fitr, marquant la fin du mois béni de -------.  Le Président de la République prie, avec la Nation, pour un Sénégal de paix, de prospérité et de solidarité et se félicite particulièrement du bon déroulement, dans la communion, des célébrations, sur l’étendue du territoire national. Revenant sur la célébration de la fête internationale du travail 2022, le Chef de l’Etat a informé le Conseil, avoir reçu, hier, en présence des membres du Gouvernement, les centrales syndicales à l’occasion de la remise des cahiers de doléances. Le -------------------------- salue, encore une fois, le rôle majeur des travailleuses et trava